In [3]:
from pymongo import MongoClient
import os 
from dotenv import load_dotenv

In [ ]:
load_dotenv()


True

In [5]:
db_url=os.getenv("DB_URL")

In [112]:
client=MongoClient(db_url)
db_name=client["Toyota_cars"]
collection=db_name["listings"]

In [20]:
def set_primary_key(collection):
    # Count documents where damage_label exists
    count = collection.count_documents({"damage_labels": {"$exists": True}})
    print(f"Total fields having damage labels are: {count}")

    # Get all matching documents in stable order
    all_docs = collection.find(
        {"damage_labels": {"$exists": True}},   # filter
        {"_id": 1}                             # projection
    ).sort("_id", 1)

    # Assign primary_key sequentially
    counter = 1
    for doc in all_docs:
        collection.update_one(
            {"_id": doc["_id"]},
            {"$set": {"primary_key": counter}}
        )
        counter += 1

    print("Primary keys assigned successfully.")


In [21]:
set_primary_key(collection)

Total fields having damage labels are: 984
Primary keys assigned successfully.


In [22]:
def get_exterior_counts(collection):
    counts=[]
    cursor = collection.find(
        {"primary_key": {"$exists": True}},
        {"exterior_images": 1, "_id": 0}
    ).sort("primary_key", 1)
    for doc in cursor:
        counts.append(len(doc.get("exterior_images", [])))

    return counts

In [24]:
counts=get_exterior_counts(collection)

In [25]:
print(counts)

[6, 10, 6, 4, 8, 4, 8, 6, 0, 10, 6, 8, 5, 11, 5, 8, 6, 9, 5, 6, 2, 3, 9, 10, 6, 6, 8, 9, 2, 6, 6, 6, 7, 6, 6, 6, 8, 9, 10, 8, 6, 11, 10, 7, 6, 6, 3, 8, 8, 4, 7, 4, 2, 3, 3, 7, 8, 4, 7, 5, 4, 6, 10, 6, 7, 6, 9, 6, 9, 10, 6, 6, 7, 9, 8, 9, 7, 6, 7, 4, 5, 4, 14, 2, 5, 7, 7, 7, 7, 7, 5, 6, 4, 6, 6, 6, 10, 6, 10, 7, 7, 4, 6, 8, 5, 2, 5, 4, 4, 7, 6, 10, 8, 5, 8, 10, 6, 9, 6, 3, 3, 6, 7, 6, 8, 8, 10, 6, 7, 6, 6, 4, 7, 5, 6, 4, 6, 6, 6, 6, 2, 3, 6, 6, 3, 7, 5, 6, 6, 6, 13, 9, 10, 5, 12, 6, 6, 1, 6, 6, 10, 7, 4, 6, 7, 5, 6, 11, 6, 6, 3, 5, 4, 6, 3, 5, 5, 6, 2, 6, 7, 6, 7, 13, 6, 7, 4, 5, 6, 7, 4, 3, 6, 7, 7, 4, 6, 6, 4, 6, 4, 9, 5, 4, 6, 6, 5, 8, 8, 8, 10, 7, 7, 3, 6, 2, 7, 3, 4, 6, 6, 6, 3, 11, 4, 3, 6, 3, 6, 6, 6, 7, 6, 8, 6, 5, 5, 6, 6, 13, 6, 6, 1, 8, 4, 6, 6, 7, 5, 4, 4, 6, 9, 8, 5, 7, 2, 4, 12, 4, 5, 6, 7, 6, 11, 7, 6, 6, 5, 10, 6, 14, 6, 4, 10, 6, 6, 6, 7, 1, 4, 3, 6, 3, 2, 6, 4, 5, 3, 8, 4, 6, 5, 4, 9, 5, 4, 3, 6, 6, 8, 8, 5, 6, 5, 6, 6, 7, 3, 6, 9, 8, 6, 6, 2, 6, 1, 8, 6, 5, 6, 4, 6, 8

In [26]:
import numpy as np

# Your array of exterior image counts
count = [5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 22, 23, 47]

# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = np.percentile(count, 25)
Q3 = np.percentile(count, 75)

# Calculate IQR
IQR = Q3 - Q1

# Calculate upper and lower fences
upper_fence = Q3 + (1.5 * IQR)
lower_fence = Q1 - (1.5 * IQR)

# Filter out outliers
non_outliers = [x for x in count if lower_fence <= x <= upper_fence]

# Get the standard (maximum non-outlier value)
standard_max_slots = max(non_outliers)

# Identify outliers
outliers = [x for x in count if x < lower_fence or x > upper_fence]

# Print results
print(f"Q1 (25th percentile): {Q1}")
print(f"Q3 (75th percentile): {Q3}")
print(f"IQR: {IQR}")
print(f"Lower Fence: {lower_fence}")
print(f"Upper Fence: {upper_fence}")
print(f"Outliers detected: {outliers}")
print(f"Non-outliers: {non_outliers}")
print(f"\n✅ USE THIS AS STANDARD: {standard_max_slots} slots")
print(f"Coverage: {len(non_outliers)}/{len(count)} = {len(non_outliers)*100/len(count):.1f}%")

Q1 (25th percentile): 9.75
Q3 (75th percentile): 18.25
IQR: 8.5
Lower Fence: -3.0
Upper Fence: 31.0
Outliers detected: [47]
Non-outliers: [5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 22, 23]

✅ USE THIS AS STANDARD: 23 slots
Coverage: 15/16 = 93.8%


In [49]:
import json

In [50]:


def export_clean_json(collection, output_file="export.json"):
    results = []

    cursor = collection.find().sort("_id", 1)  # Ascending order

    for doc in cursor:
        row = {
            "_id": str(doc["_id"])  # convert ObjectId to string
        }

        # Pull damage_labels_numerical safely
        damage = doc.get("damage_labels_numerical", {})

        # Add all fields from damage_labels_numerical
        for key in [
            "Normal", "crack", "dent", "glass_shatter",
            "lamp_broken", "scratch", "tire_flat", "total_images"
        ]:
            row[key] = damage.get(key, None)

        # Add optional fields
        row["description_rating"] = doc.get("description_rating", None)
        row["rating"] = doc.get("rating", None)

        results.append(row)

    # Save to JSON
    with open(output_file, "w") as f:
        json.dump(results, f, indent=4)

    print(f"✅ Exported {len(results)} records to {output_file}")


In [111]:
import csv

def export_upto_primary_key_to_csv(collection, filename="Toyota_cars.csv"):
    try:
        last_key = int(input("Enter last primary_key value: "))
    except ValueError:
        print("Invalid number")
        return

    cursor = collection.find(
        {"primary_key": {"$lte": last_key}}
    ).sort("primary_key", 1)

    fields = [
        "primary_key",
        "normal", "crack", "dent", "glass_shatter",
        "lamp_broken", "scratch", "tire_flat",
        "total_images", "description_rating", "rating", "condition"
    ]

    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(fields)

        count = 0

        for doc in cursor:
            damage = doc.get("damage_labels_numerical", {})

            row = [
                doc.get("primary_key"),
                damage.get("Normal", ""),
                damage.get("crack", ""),
                damage.get("dent", ""),
                damage.get("glass_shatter", ""),
                damage.get("lamp_broken", ""),
                damage.get("scratch", ""),
                damage.get("tire_flat", ""),
                damage.get("total_images", ""),
                doc.get("description_rating", ""),
                doc.get("rating", ""),
                ""  # condition empty
            ]

            writer.writerow(row)
            count += 1

    print(f"✅ Exported {count} records (1 → {last_key}) to {filename}")


In [113]:
def analyze_primary_keys(collection):
    # Count all documents
    total_docs = collection.count_documents({})
    
    # Count documents that have primary_key
    total_with_key = collection.count_documents({"primary_key": {"$exists": True}})

    # Get smallest primary_key
    min_doc = collection.find_one(
        {"primary_key": {"$exists": True}},
        sort=[("primary_key", 1)]
    )

    # Get largest primary_key
    max_doc = collection.find_one(
        {"primary_key": {"$exists": True}},
        sort=[("primary_key", -1)]
    )

    print("📊 Primary Key Analysis")
    print("-" * 40)
    print(f"Total documents: {total_docs}")
    print(f"Documents with primary_key: {total_with_key}")

    if min_doc and max_doc:
        print(f"Minimum primary_key: {min_doc['primary_key']}")
        print(f"Maximum primary_key: {max_doc['primary_key']}")
    else:
        print("No primary_key field found.")

    return {
        "total_documents": total_docs,
        "with_primary_key": total_with_key,
        "min_primary_key": min_doc["primary_key"] if min_doc else None,
        "max_primary_key": max_doc["primary_key"] if max_doc else None
    }


In [114]:
analyze_primary_keys(collection)

📊 Primary Key Analysis
----------------------------------------
Total documents: 2692
Documents with primary_key: 602
Minimum primary_key: 1
Maximum primary_key: 602


{'total_documents': 2692,
 'with_primary_key': 602,
 'min_primary_key': 1,
 'max_primary_key': 602}

In [115]:
export_upto_primary_key_to_csv(collection)

✅ Exported 602 records (1 → 602) to Toyota_cars.csv
